## Adapted from Victor Zhou's CNN published at [this blog post](https://victorzhou.com/blog/keras-cnn-tutorial/).

In [21]:
import math
import mnist
import numpy as np

# Using tf.keras for Ghost Batch Norm capability.
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Nadam

import tensorflow as tf
#print(tf.version)
# Disable INFO and WARNING messages from TensorFlow.
# Our Keras version (2.2.4 / TF 1.15.0) throws deprecation warnings.
tf.logging.set_verbosity(tf.logging.ERROR)

In [22]:
train_images = mnist.train_images() 
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

num_filters = 8
filter_size = 3
pool_size = 2

In [23]:
# Build the model.
batch_size = 100       # TODO: make this less arbitrary.
ghost_batch_size = 10  # TODO: make this less arbitrary.

model = Sequential([
    # The authors required ghost batch normalization in their experiments.
    # The papers they reference use normalization preceding each convolutional layer.
    BatchNormalization(virtual_batch_size=ghost_batch_size),
    Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(units=10, activation='softmax'),
])

In [24]:
# Adapted from https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
def step_lr_decay(epoch):
    # This starts with a learning rate higher than the default learning rate in
    # Vanilla SGD. It drops by a factor of 5 each step, and I chose to drop after
    # every other epoch to make training not take too long on a laptop.
    # Think of this as a half-life equation.
    initial_lrate=0.15
    drop=0.2
    epochs_drop=2.0
    current_rate = initial_lrate * math.pow(drop, math.floor(epoch / epochs_drop))
    print("Epoch %d learning rate: %f" % (epoch + 1, current_rate))
    return current_rate

In [25]:
# Compile the model.
epochs = 6            # Run enough epochs to test the training schedule.
learning_rate = 0.00  # Will be overrided by the scheduler.
decay_rate = 0.00     # Default decay rate.
momentum = 0.0        # Default momentum.

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

## Experiment 1: Vanilla SGD, decaying learning rate.

In [26]:
# Train the model.
decay_lr_scheduler = LearningRateScheduler(step_lr_decay)

exp1_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[decay_lr_scheduler]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1 learning rate: 0.150000
Epoch 1/6
60000/60000 [==============================] - 6s 95us/sample - loss: 0.2490 - acc: 0.9255 - val_loss: 0.1272 - val_acc: 0.9601
Epoch 2 learning rate: 0.150000
Epoch 2/6
60000/60000 [==============================] - 6s 95us/sample - loss: 0.1177 - acc: 0.9642 - val_loss: 0.1123 - val_acc: 0.9644
Epoch 3 learning rate: 0.030000
Epoch 3/6
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0734 - acc: 0.9790 - val_loss: 0.0746 - val_acc: 0.9774
Epoch 4 learning rate: 0.030000
Epoch 4/6
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0654 - acc: 0.9807 - val_loss: 0.0749 - val_acc: 0.9763
Epoch 5 learning rate: 0.006000
Epoch 5/6
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0577 - acc: 0.9833 - val_loss: 0.0713 - val_acc: 0.9775
Epoch 6 learning rate: 0.006000
Epoch 6/6
60000/60000 [==============================] - 6s 93us/sam

## Experiment 2: Vanilla SGD, increasing batch size.

In [27]:
# Keras has no built in for this. 
# Code is from https://www.codementor.io/nitinsurya/how-to-re-initialize-keras-model-weights-et41zre2g
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [28]:
reset_weights(model)
learning_rate = 0.07  # Set a new constant.
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Just run each step independently. Keras has no callback for this.
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 25,
    validation_data=(test_images, to_categorical(test_labels))
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 96us/sample - loss: 0.2630 - acc: 0.9213 - val_loss: 0.1394 - val_acc: 0.9582
Epoch 2/2
60000/60000 [==============================] - 6s 95us/sample - loss: 0.1217 - acc: 0.9646 - val_loss: 0.1083 - val_acc: 0.9657
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0821 - acc: 0.9761 - val_loss: 0.0840 - val_acc: 0.9739
Epoch 2/2
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0753 - acc: 0.9785 - val_loss: 0.0823 - val_acc: 0.9735
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0687 - acc: 0.9809 - val_loss: 0.0798 - val_acc: 0.9745
Epoch 2/2
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0674 - acc: 0.9812 - val_loss: 0.0797 - val_acc: 0.9752


## Experiment 3: Vanilla SGD, hybrid.

In [29]:
def hybrid_lr_decay(epoch):
    # Start off from the constant learning rate from the hybrid's initialization.
    initial_lrate=0.05
    drop=0.2
    epochs_drop=2.0
    current_rate = initial_lrate * math.pow(drop, math.floor(epoch / epochs_drop))
    print("Epoch %d learning rate: %f" % (epoch + 1, current_rate))
    return current_rate

hybrid_lr_scheduler = LearningRateScheduler(hybrid_lr_decay)

In [30]:
reset_weights(model)
learning_rate = 0.05  # Set a new constant.
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(
    optimizer=sgd,  # Vanilla SGD experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Run a step, and then increase the batch size manually. The decay scheduler
# will decay the learning rate in the next step.
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=4,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[hybrid_lr_scheduler]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 93us/sample - loss: 0.2621 - acc: 0.9235 - val_loss: 0.1302 - val_acc: 0.9624
Epoch 2/2
60000/60000 [==============================] - 6s 98us/sample - loss: 0.1180 - acc: 0.9658 - val_loss: 0.1001 - val_acc: 0.9687
Train on 60000 samples, validate on 10000 samples
Epoch 1 learning rate: 0.050000
Epoch 1/4
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0807 - acc: 0.9767 - val_loss: 0.0800 - val_acc: 0.9764
Epoch 2 learning rate: 0.050000
Epoch 2/4
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0755 - acc: 0.9782 - val_loss: 0.0789 - val_acc: 0.9768
Epoch 3 learning rate: 0.010000
Epoch 3/4
60000/60000 [==============================] - 6s 92us/sample - loss: 0.0696 - acc: 0.9801 - val_loss: 0.0772 - val_acc: 0.9775
Epoch 4 learning rate: 0.010000
Epoch 4/4
60000/60000 [==============================] - 5s 90us/sample - loss: 0.

## Experiment 4: Adam, decaying learning rate

In [36]:
reset_weights(model)
nadam = Nadam() #The paper uses the default settings for Adam

model.compile(
    optimizer=nadam,  # Adam experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Train the model.
decay_lr_scheduler = LearningRateScheduler(step_lr_decay)

exp4_results = model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[decay_lr_scheduler]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1 learning rate: 0.150000
Epoch 1/6
60000/60000 [==============================] - 6s 99us/sample - loss: 3.1535 - acc: 0.8614 - val_loss: 0.2747 - val_acc: 0.9241
Epoch 2 learning rate: 0.150000
Epoch 2/6
60000/60000 [==============================] - 6s 96us/sample - loss: 0.2484 - acc: 0.9293 - val_loss: 0.1718 - val_acc: 0.9504
Epoch 3 learning rate: 0.030000
Epoch 3/6
60000/60000 [==============================] - 6s 97us/sample - loss: 0.1397 - acc: 0.9605 - val_loss: 0.1351 - val_acc: 0.9609
Epoch 4 learning rate: 0.030000
Epoch 4/6
60000/60000 [==============================] - 6s 97us/sample - loss: 0.1185 - acc: 0.9656 - val_loss: 0.1149 - val_acc: 0.9660
Epoch 5 learning rate: 0.006000
Epoch 5/6
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0931 - acc: 0.9731 - val_loss: 0.1052 - val_acc: 0.9691
Epoch 6 learning rate: 0.006000
Epoch 6/6
60000/60000 [==============================] - 6s 97us/sam

## Experiment 5: Adam, increasing batch size.

In [37]:
reset_weights(model)
nadam = Nadam() #The paper uses the default settings for Adam

model.compile(
    optimizer=nadam,  # Adam experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Just run each step independently. Keras has no callback for this.
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size * 25,
    validation_data=(test_images, to_categorical(test_labels))
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 101us/sample - loss: 0.5175 - acc: 0.8552 - val_loss: 0.2535 - val_acc: 0.9270
Epoch 2/2
60000/60000 [==============================] - 6s 96us/sample - loss: 0.2282 - acc: 0.9344 - val_loss: 0.1831 - val_acc: 0.9488
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 88us/sample - loss: 0.1749 - acc: 0.9512 - val_loss: 0.1677 - val_acc: 0.9545
Epoch 2/2
60000/60000 [==============================] - 5s 89us/sample - loss: 0.1626 - acc: 0.9548 - val_loss: 0.1592 - val_acc: 0.9563
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 80us/sample - loss: 0.1530 - acc: 0.9566 - val_loss: 0.1553 - val_acc: 0.9570
Epoch 2/2
60000/60000 [==============================] - 5s 78us/sample - loss: 0.1510 - acc: 0.9581 - val_loss: 0.1532 - val_acc: 0.9577


## Experiment 6: Adam, hybrid.

In [38]:
def hybrid_lr_decay_adam(epoch):
    # Start off from the constant learning rate from the hybrid's initialization.
    initial_lrate=0.001 #initial learning rate for Adam's Keras's default = 0.001
    drop=0.2
    epochs_drop=2.0
    current_rate = initial_lrate * math.pow(drop, math.floor(epoch / epochs_drop))
    print("Epoch %d learning rate: %f" % (epoch + 1, current_rate))
    return current_rate

hybrid_lr_scheduler_adam = LearningRateScheduler(hybrid_lr_decay_adam)

In [39]:
reset_weights(model)
nadam = Nadam() #The paper uses the default settings for Adam

model.compile(
    optimizer=nadam,  # Adam experiment.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Run a step, and then increase the batch size manually. The decay scheduler
# will decay the learning rate in the next step.
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=2,
    batch_size=batch_size,
    validation_data=(test_images, to_categorical(test_labels))
)
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=4,
    batch_size=batch_size * 5,
    validation_data=(test_images, to_categorical(test_labels)),
    callbacks=[hybrid_lr_scheduler_adam]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 6s 104us/sample - loss: 0.4084 - acc: 0.8821 - val_loss: 0.2391 - val_acc: 0.9303
Epoch 2/2
60000/60000 [==============================] - 6s 100us/sample - loss: 0.2087 - acc: 0.9410 - val_loss: 0.1620 - val_acc: 0.9548
Train on 60000 samples, validate on 10000 samples
Epoch 1 learning rate: 0.001000
Epoch 1/4
60000/60000 [==============================] - 5s 90us/sample - loss: 0.1518 - acc: 0.9576 - val_loss: 0.1431 - val_acc: 0.9579
Epoch 2 learning rate: 0.001000
Epoch 2/4
60000/60000 [==============================] - 5s 90us/sample - loss: 0.1396 - acc: 0.9606 - val_loss: 0.1327 - val_acc: 0.9608
Epoch 3 learning rate: 0.000200
Epoch 3/4
60000/60000 [==============================] - 5s 90us/sample - loss: 0.1303 - acc: 0.9637 - val_loss: 0.1311 - val_acc: 0.9613
Epoch 4 learning rate: 0.000200
Epoch 4/4
60000/60000 [==============================] - 5s 90us/sample - loss: 